# 🌱 Function 2: Calculate Vegetation Indices

## Building the `calculate_vegetation_indices` Function

**Learning Objectives:**
- Master vegetation index calculations from multispectral satellite imagery
- Understand the mathematical foundations of spectral indices (NDVI, EVI, SAVI)
- Learn to handle different satellite sensor configurations
- Implement robust band arithmetic and data validation
- Create time-series vegetation monitoring workflows
- Handle atmospheric correction and quality assessment

**Professional Context:**
Vegetation indices are fundamental tools in remote sensing and environmental monitoring. Professionals use these techniques for:
- Agricultural monitoring and precision farming
- Forest health assessment and deforestation tracking
- Drought monitoring and water stress detection
- Climate change impact assessment
- Ecosystem health and biodiversity monitoring

## 🎯 Function Overview

**Function Signature:**
```python
def calculate_vegetation_indices(image_path, indices=['ndvi'], 
                               band_config=None, output_dir=None, 
                               mask_invalid=True, scale_factor=1.0):
    """
    Calculate multiple vegetation indices from multispectral satellite imagery.
    
    Parameters:
    -----------
    image_path : str
        Path to the multispectral satellite image
    indices : list, optional
        List of indices to calculate: ['ndvi', 'evi', 'savi', 'ndwi', 'gndvi', 'arvi']
        Default: ['ndvi']
    band_config : dict, optional
        Band configuration mapping. If None, assumes standard Landsat/Sentinel order
        Example: {'red': 4, 'nir': 5, 'blue': 2, 'green': 3, 'swir1': 6}
    output_dir : str, optional
        Directory to save output index rasters. If None, returns arrays only
    mask_invalid : bool, optional
        Whether to mask invalid/nodata values, Default: True
    scale_factor : float, optional
        Scale factor for reflectance values (e.g., 0.0001 for Landsat), Default: 1.0
    
    Returns:
    --------
    dict
        Dictionary containing calculated indices, statistics, and metadata
    """
```

**Key Capabilities:**
- 📊 Calculate multiple vegetation indices (NDVI, EVI, SAVI, NDWI, etc.)
- 🛰️ Support various satellite sensors (Landsat, Sentinel-2, etc.)
- 📈 Generate comprehensive statistics and quality metrics
- 💾 Export results as GeoTIFF files with proper metadata
- 🔍 Handle invalid data and atmospheric effects
- 📋 Provide detailed analysis reports and summaries

## 🏗️ Implementation Strategy

Our vegetation indices function will follow this workflow:

### Step 1: Data Loading and Validation
```python
# Load multispectral data and validate band configuration
with rasterio.open(image_path) as src:
    bands_data = src.read()
    profile = src.profile
    
# Apply scale factor for reflectance conversion
bands_data = bands_data * scale_factor
```

### Step 2: Index Calculations
```python
# NDVI calculation
def calculate_ndvi(red, nir):
    return (nir - red) / (nir + red)

# EVI calculation
def calculate_evi(red, nir, blue, L=1, C1=6, C2=7.5, G=2.5):
    return G * (nir - red) / (nir + C1 * red - C2 * blue + L)
```

### Step 3: Quality Assessment and Statistics
```python
# Calculate statistics for each index
statistics = {
    'mean': np.nanmean(index_data),
    'std': np.nanstd(index_data),
    'min': np.nanmin(index_data),
    'max': np.nanmax(index_data)
}
```

## 🚀 Hands-On Example: Building the Function

Let's build the complete vegetation indices function step by step:

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

def calculate_vegetation_indices(image_path, indices=['ndvi'], 
                               band_config=None, output_dir=None, 
                               mask_invalid=True, scale_factor=1.0):
    """
    Calculate multiple vegetation indices from multispectral satellite imagery.
    """
    
    print(f"🌱 Starting vegetation indices calculation...")
    print(f"   📁 Input image: {os.path.basename(image_path)}")
    print(f"   📊 Indices to calculate: {indices}")
    
    # Step 1: Input validation
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")
    
    # Define available indices
    available_indices = ['ndvi', 'evi', 'savi', 'ndwi', 'gndvi', 'arvi', 'msavi']
    
    # Validate requested indices
    invalid_indices = [idx for idx in indices if idx.lower() not in available_indices]
    if invalid_indices:
        raise ValueError(f"Invalid indices: {invalid_indices}. Available: {available_indices}")
    
    indices = [idx.lower() for idx in indices]  # Normalize to lowercase
    
    # Step 2: Load multispectral data
    with rasterio.open(image_path) as src:
        bands_data = src.read().astype(np.float32)
        profile = src.profile.copy()
        transform = src.transform
        crs = src.crs
        nodata = src.nodata
        
        print(f"   📊 Image info:")
        print(f"     Shape: {bands_data.shape}")
        print(f"     Bands: {src.count}")
        print(f"     CRS: {crs}")
        print(f"     NoData: {nodata}")
    
    # Step 3: Apply scale factor for reflectance conversion
    if scale_factor != 1.0:
        print(f"   🔧 Applying scale factor: {scale_factor}")
        bands_data = bands_data * scale_factor
    
    # Step 4: Set up band configuration
    if band_config is None:
        # Default configuration (Landsat-like: B,G,R,NIR,SWIR1,SWIR2)
        if bands_data.shape[0] >= 6:
            band_config = {
                'blue': 1, 'green': 2, 'red': 3, 'nir': 4, 'swir1': 5, 'swir2': 6
            }
        elif bands_data.shape[0] >= 4:
            band_config = {
                'blue': 1, 'green': 2, 'red': 3, 'nir': 4
            }
        else:
            raise ValueError("Insufficient bands for vegetation index calculation")
    
    print(f"   🎛️ Band configuration: {band_config}")
    
    # Step 5: Extract individual bands (convert to 0-based indexing)
    bands = {}
    for band_name, band_num in band_config.items():
        if band_num <= bands_data.shape[0]:
            bands[band_name] = bands_data[band_num - 1]  # Convert to 0-based
        else:
            print(f"   ⚠️ Warning: Band {band_num} ({band_name}) not available")
    
    # Step 6: Mask invalid data
    if mask_invalid:
        for band_name, band_data in bands.items():
            if nodata is not None:
                bands[band_name] = np.ma.masked_equal(band_data, nodata)
            else:
                bands[band_name] = np.ma.masked_invalid(band_data)
    
    # Step 7: Calculate vegetation indices
    results = {}
    calculated_indices = {}
    
    for index_name in indices:
        print(f"   🧮 Calculating {index_name.upper()}...")
        
        try:
            if index_name == 'ndvi':
                # Normalized Difference Vegetation Index
                if 'red' in bands and 'nir' in bands:
                    red, nir = bands['red'], bands['nir']
                    index_data = (nir - red) / (nir + red + 1e-8)  # Add small epsilon
                else:
                    raise ValueError("NDVI requires red and NIR bands")
                    
            elif index_name == 'evi':
                # Enhanced Vegetation Index
                if all(b in bands for b in ['red', 'nir', 'blue']):
                    red, nir, blue = bands['red'], bands['nir'], bands['blue']
                    G, C1, C2, L = 2.5, 6.0, 7.5, 1.0
                    index_data = G * (nir - red) / (nir + C1 * red - C2 * blue + L + 1e-8)
                else:
                    raise ValueError("EVI requires red, NIR, and blue bands")
                    
            elif index_name == 'savi':
                # Soil Adjusted Vegetation Index
                if 'red' in bands and 'nir' in bands:
                    red, nir = bands['red'], bands['nir']
                    L = 0.5  # Soil brightness correction factor
                    index_data = ((nir - red) / (nir + red + L)) * (1 + L)
                else:
                    raise ValueError("SAVI requires red and NIR bands")
                    
            elif index_name == 'ndwi':
                # Normalized Difference Water Index
                if 'green' in bands and 'nir' in bands:
                    green, nir = bands['green'], bands['nir']
                    index_data = (green - nir) / (green + nir + 1e-8)
                else:
                    raise ValueError("NDWI requires green and NIR bands")
                    
            elif index_name == 'gndvi':
                # Green Normalized Difference Vegetation Index
                if 'green' in bands and 'nir' in bands:
                    green, nir = bands['green'], bands['nir']
                    index_data = (nir - green) / (nir + green + 1e-8)
                else:
                    raise ValueError("GNDVI requires green and NIR bands")
                    
            elif index_name == 'arvi':
                # Atmospherically Resistant Vegetation Index
                if all(b in bands for b in ['red', 'nir', 'blue']):
                    red, nir, blue = bands['red'], bands['nir'], bands['blue']
                    # ARVI formula: (NIR - (2*Red - Blue)) / (NIR + (2*Red - Blue))
                    rb = 2 * red - blue
                    index_data = (nir - rb) / (nir + rb + 1e-8)
                else:
                    raise ValueError("ARVI requires red, NIR, and blue bands")
                    
            elif index_name == 'msavi':
                # Modified Soil Adjusted Vegetation Index
                if 'red' in bands and 'nir' in bands:
                    red, nir = bands['red'], bands['nir']
                    # MSAVI formula
                    index_data = (2 * nir + 1 - np.sqrt((2 * nir + 1)**2 - 8 * (nir - red))) / 2
                else:
                    raise ValueError("MSAVI requires red and NIR bands")
            
            # Clip values to reasonable ranges for vegetation indices
            if index_name in ['ndvi', 'evi', 'savi', 'gndvi', 'arvi', 'msavi']:
                index_data = np.clip(index_data, -1, 1)
            elif index_name == 'ndwi':
                index_data = np.clip(index_data, -1, 1)
                
            calculated_indices[index_name] = index_data
            
        except Exception as e:
            print(f"   ❌ Failed to calculate {index_name.upper()}: {e}")
            continue
    
    # Step 8: Calculate statistics for each index
    print(f"   📈 Calculating statistics...")
    
    for index_name, index_data in calculated_indices.items():
        # Get valid data (not masked)
        if hasattr(index_data, 'compressed'):
            valid_data = index_data.compressed()
        else:
            valid_data = index_data[~np.isnan(index_data)]
        
        if len(valid_data) > 0:
            statistics = {
                'mean': float(np.mean(valid_data)),
                'median': float(np.median(valid_data)),
                'std': float(np.std(valid_data)),
                'min': float(np.min(valid_data)),
                'max': float(np.max(valid_data)),
                'percentile_25': float(np.percentile(valid_data, 25)),
                'percentile_75': float(np.percentile(valid_data, 75)),
                'valid_pixels': len(valid_data),
                'total_pixels': index_data.size,
                'valid_percentage': (len(valid_data) / index_data.size) * 100
            }
        else:
            statistics = {k: None for k in ['mean', 'median', 'std', 'min', 'max', 
                                          'percentile_25', 'percentile_75']}
            statistics.update({'valid_pixels': 0, 'total_pixels': index_data.size, 'valid_percentage': 0})
        
        results[index_name] = {
            'data': index_data,
            'statistics': statistics
        }
    
    # Step 9: Save outputs if directory specified
    if output_dir:
        print(f"   💾 Saving indices to: {output_dir}")
        os.makedirs(output_dir, exist_ok=True)
        
        # Update profile for single-band output
        output_profile = profile.copy()
        output_profile.update({
            'count': 1,
            'dtype': 'float32',
            'nodata': -9999
        })
        
        for index_name, result in results.items():
            output_path = os.path.join(output_dir, f"{index_name}.tif")
            
            # Convert masked array to regular array with nodata values
            output_data = result['data'].copy()
            if hasattr(output_data, 'filled'):
                output_data = output_data.filled(-9999)
            else:
                output_data[np.isnan(output_data)] = -9999
            
            with rasterio.open(output_path, 'w', **output_profile) as dst:
                dst.write(output_data, 1)
                
            results[index_name]['output_path'] = output_path
    
    # Step 10: Compile comprehensive results
    final_results = {
        'calculation_successful': True,
        'indices_calculated': list(results.keys()),
        'results': results,
        'input_metadata': {
            'image_path': image_path,
            'image_shape': bands_data.shape,
            'crs': str(crs),
            'transform': transform,
            'band_configuration': band_config
        },
        'processing_parameters': {
            'indices_requested': indices,
            'scale_factor': scale_factor,
            'mask_invalid': mask_invalid,
            'output_directory': output_dir
        }
    }
    
    # Print summary
    print(f"\n🎉 Vegetation indices calculation complete!")
    print(f"   ✅ Successfully calculated: {len(results)} indices")
    for index_name, result in results.items():
        stats = result['statistics']
        if stats['mean'] is not None:
            print(f"   📊 {index_name.upper()}: mean={stats['mean']:.3f}, std={stats['std']:.3f}, valid={stats['valid_percentage']:.1f}%")
    
    return final_results

## 🧪 Test the Function

Let's test our vegetation indices function with synthetic multispectral data:

In [ ]:
# Create synthetic multispectral satellite data for testing
print("🏗️ Creating synthetic multispectral satellite data...\n")

import tempfile

# Image dimensions
height, width = 200, 300
n_bands = 6

# Create coordinate system
x = np.linspace(-120, -119, width)
y = np.linspace(37, 38, height)
X, Y = np.meshgrid(x, y)

# Simulate different land cover types with realistic reflectance values
# Band 1: Blue (0.45-0.52 μm)
blue = 0.05 + 0.03 * np.random.random((height, width)) + \
       0.02 * np.sin(X * 20) * np.cos(Y * 15)

# Band 2: Green (0.52-0.60 μm)
green = 0.08 + 0.04 * np.random.random((height, width)) + \
        0.03 * np.exp(-((X + 119.5)**2 + (Y - 37.5)**2) * 50)

# Band 3: Red (0.63-0.69 μm) - absorbed by healthy vegetation
red = 0.04 + 0.02 * np.random.random((height, width)) + \
      0.08 * np.exp(-((X + 119.2)**2 + (Y - 37.8)**2) * 30)  # Urban/bare soil

# Band 4: NIR (0.77-0.90 μm) - highly reflected by healthy vegetation
vegetation_mask = ((X + 119.5)**2 + (Y - 37.5)**2) < 0.1  # Dense vegetation area
nir = np.where(vegetation_mask, 
               0.4 + 0.1 * np.random.random((height, width)),  # High NIR for vegetation
               0.15 + 0.05 * np.random.random((height, width)))  # Lower NIR for other areas

# Band 5: SWIR1 (1.55-1.75 μm)
swir1 = 0.12 + 0.08 * np.random.random((height, width)) + \
        0.05 * np.cos(X * 10) * np.sin(Y * 8)

# Band 6: SWIR2 (2.08-2.35 μm)
swir2 = 0.08 + 0.06 * np.random.random((height, width))

# Stack bands (following Landsat order)
multispectral_data = np.stack([blue, green, red, nir, swir1, swir2])

# Add some nodata areas (clouds, water, etc.)
cloud_mask = np.random.random((height, width)) < 0.05  # 5% clouds
for i in range(n_bands):
    multispectral_data[i][cloud_mask] = -9999

# Define geospatial properties
transform = rasterio.transform.from_bounds(-120, 37, -119, 38, width, height)
crs = 'EPSG:4326'

# Save synthetic satellite image
test_image_path = '/tmp/synthetic_satellite.tif'
profile = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': n_bands,
    'dtype': 'float32',
    'crs': crs,
    'transform': transform,
    'nodata': -9999
}

with rasterio.open(test_image_path, 'w', **profile) as dst:
    for i in range(n_bands):
        dst.write(multispectral_data[i].astype(np.float32), i + 1)

print(f"✅ Created synthetic satellite image: {test_image_path}")
print(f"   📊 Shape: {n_bands} bands x {height} x {width} pixels")
print(f"   📏 Bounds: {rasterio.transform.array_bounds(height, width, transform)}")
print(f"   🌱 Simulated vegetation area around (-119.5, 37.5)")
print(f"   ☁️ Added {cloud_mask.sum()} cloud pixels")

# Visualize the synthetic data
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
band_names = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']
cmaps = ['Blues', 'Greens', 'Reds', 'YlOrRd', 'copper', 'gray']

for i, (ax, name, cmap) in enumerate(zip(axes.flat, band_names, cmaps)):
    band_data = multispectral_data[i]
    band_data_display = np.where(band_data == -9999, np.nan, band_data)
    
    im = ax.imshow(band_data_display, extent=[-120, -119, 37, 38], 
                   cmap=cmap, aspect='auto')
    ax.set_title(f'{name} Band (Band {i+1})')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.colorbar(im, ax=ax, shrink=0.8)

plt.tight_layout()
plt.show()

print("\n📋 Synthetic satellite data ready for vegetation index analysis!")

In [ ]:
# Test 1: Calculate multiple vegetation indices
print("🚀 Test 1: Calculate Multiple Vegetation Indices\n")

# Calculate common vegetation indices
indices_to_calculate = ['ndvi', 'evi', 'savi', 'ndwi', 'gndvi']

results = calculate_vegetation_indices(
    image_path=test_image_path,
    indices=indices_to_calculate,
    output_dir='/tmp/vegetation_indices/',
    mask_invalid=True,
    scale_factor=1.0  # Data already in reflectance units
)

# Display results summary
print(f"\n📊 Vegetation Indices Results Summary:")
print(f"   Successfully calculated: {len(results['indices_calculated'])} indices")
print(f"   Indices: {', '.join([idx.upper() for idx in results['indices_calculated']])}")

# Show statistics for each index
print(f"\n📈 Index Statistics:")
for index_name, result in results['results'].items():
    stats = result['statistics']
    if stats['mean'] is not None:
        print(f"   {index_name.upper()}:")
        print(f"     Range: {stats['min']:.3f} to {stats['max']:.3f}")
        print(f"     Mean: {stats['mean']:.3f} ± {stats['std']:.3f}")
        print(f"     Valid data: {stats['valid_percentage']:.1f}%")

# Visualize the calculated indices
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Define color maps for each index
index_cmaps = {
    'ndvi': 'RdYlGn',
    'evi': 'viridis',
    'savi': 'RdYlGn',
    'ndwi': 'Blues',
    'gndvi': 'YlGn'
}

for i, (index_name, result) in enumerate(results['results'].items()):
    if i >= len(axes):
        break
        
    index_data = result['data']
    stats = result['statistics']
    
    # Create display array (handle masked data)
    if hasattr(index_data, 'filled'):
        display_data = np.where(index_data.mask, np.nan, index_data.data)
    else:
        display_data = np.where(index_data == -9999, np.nan, index_data)
    
    # Plot index
    cmap = index_cmaps.get(index_name, 'viridis')
    im = axes[i].imshow(display_data, extent=[-120, -119, 37, 38], 
                       cmap=cmap, aspect='auto')
    
    axes[i].set_title(f'{index_name.upper()}\nMean: {stats["mean"]:.3f}')
    axes[i].set_xlabel('Longitude')
    axes[i].set_ylabel('Latitude')
    
    # Add colorbar
    plt.colorbar(im, ax=axes[i], shrink=0.8)

# Hide unused subplot
if len(results['results']) < len(axes):
    axes[-1].set_visible(False)

plt.tight_layout()
plt.show()

print(f"\n✅ Vegetation indices calculation and visualization complete!")

## 💡 Understanding Vegetation Indices

### Key Concepts:

**NDVI (Normalized Difference Vegetation Index):**
- **Formula**: (NIR - Red) / (NIR + Red)
- **Range**: -1 to +1
- **Applications**: General vegetation health, biomass estimation
- **Interpretation**: Values > 0.3 indicate healthy vegetation

**EVI (Enhanced Vegetation Index):**
- **Formula**: G × (NIR - Red) / (NIR + C1×Red - C2×Blue + L)
- **Advantages**: Better performance in high biomass areas, atmospheric correction
- **Applications**: Forest monitoring, agricultural productivity

**SAVI (Soil Adjusted Vegetation Index):**
- **Formula**: ((NIR - Red) / (NIR + Red + L)) × (1 + L)
- **Purpose**: Reduces soil background effects
- **Applications**: Sparse vegetation, arid environments

**NDWI (Normalized Difference Water Index):**
- **Formula**: (Green - NIR) / (Green + NIR)
- **Purpose**: Water content detection, drought monitoring
- **Applications**: Irrigation management, water stress assessment

### Interpretation Guidelines:
- **NDVI < 0**: Water, clouds, snow, bare rock
- **NDVI 0-0.1**: Bare soil, urban areas
- **NDVI 0.1-0.3**: Sparse vegetation, grasslands
- **NDVI 0.3-0.6**: Moderate vegetation, crops
- **NDVI > 0.6**: Dense vegetation, forests

## 🎯 Your Task: Implement and Test

**Requirements:**
1. **Implement the function** exactly as shown above
2. **Handle multiple satellite sensors** (Landsat, Sentinel-2, etc.)
3. **Support flexible band configurations** for different sensor types
4. **Apply appropriate scaling** for different reflectance formats
5. **Validate mathematical formulas** and handle edge cases
6. **Generate comprehensive outputs** with statistics and visualizations

**Key Implementation Points:**
- Use robust mathematical formulations to avoid division by zero
- Handle nodata values and invalid reflectance ranges properly
- Support both scaled integer and floating-point reflectance data
- Provide meaningful progress feedback for large datasets
- Return structured results with comprehensive metadata

**Testing Strategy:**
```python
# Test different scenarios:
# 1. Landsat 8/9 surface reflectance data
# 2. Sentinel-2 Level-2A data
# 3. High-resolution aerial imagery
# 4. Time series analysis workflows
# 5. Different atmospheric conditions
# 6. Various land cover types
```

## 🔧 Testing Your Implementation

Run the official tests to verify your function works correctly:

```bash
cd /workspaces/your-repo
python -m pytest tests/test_advanced_rasterio_analysis.py::test_calculate_vegetation_indices -v
```

### Additional Testing Ideas:
```python
# Test with real satellite data
results = calculate_vegetation_indices(
    image_path='landsat8_scene.tif',
    indices=['ndvi', 'evi', 'savi'],
    band_config={'red': 4, 'nir': 5, 'blue': 2, 'green': 3},
    scale_factor=0.0001,  # Landsat scaling factor
    output_dir='vegetation_analysis/'
)

# Validate index ranges
for index_name, result in results['results'].items():
    stats = result['statistics']
    assert -1 <= stats['min'] <= 1
    assert -1 <= stats['max'] <= 1
```

## 📚 Professional Applications

### Real-World Use Cases:

**Agriculture:**
- Crop health monitoring and yield prediction
- Precision agriculture and variable rate applications
- Irrigation scheduling and water management
- Disease and pest detection in agricultural fields

**Environmental Monitoring:**
- Forest health assessment and deforestation tracking
- Ecosystem productivity and carbon sequestration studies
- Biodiversity monitoring and habitat assessment
- Climate change impact evaluation

**Natural Resource Management:**
- Rangeland condition assessment for livestock management
- Wetland monitoring and restoration planning
- Urban green space analysis and planning
- Invasive species detection and mapping

**Research Applications:**
- Phenology studies and seasonal vegetation patterns
- Drought monitoring and early warning systems
- Land use/land cover change detection
- Ecosystem service quantification

### Industry Standards:
- **Data Sources**: Landsat (30m), Sentinel-2 (10-20m), MODIS (250-500m)
- **Temporal Resolution**: Daily (MODIS), 5-day (Sentinel-2), 16-day (Landsat)
- **Quality Standards**: Cloud-free composites, atmospheric correction
- **Validation**: Ground-truth measurements, cross-sensor comparison

## 🚀 Next Steps

Once this function works and passes the tests, move on to:
- **Function 3**: `extract_spatial_samples()` - Learn to extract point and zonal statistics from raster data

**This completes the vegetation analysis and spectral index skills!**

## 🎓 Real-World Applications

The vegetation index techniques you've mastered are used for:
- **Precision Agriculture**: Optimizing crop management and resource allocation
- **Environmental Monitoring**: Tracking ecosystem health and biodiversity
- **Climate Research**: Understanding vegetation responses to climate change
- **Natural Resource Management**: Sustainable forest and rangeland management
- **Food Security**: Global crop monitoring and yield forecasting

**Excellent work on mastering vegetation index analysis! 🍀**